In [1]:
import pandas as pd
import re
import os


In [2]:

# Load the CSV file
file_path = "/Users/bhupendra/projects/crocus/data/flux_data/data/AmeriFlux/US-CU1_HH_202409010000_202410010000.csv"
df = pd.read_csv(file_path)
df["TIMESTAMP_START"] = pd.to_datetime(df["TIMESTAMP_START"], format="%Y%m%d%H%M")
df["TIMESTAMP_END"] = pd.to_datetime(df["TIMESTAMP_END"], format="%Y%m%d%H%M")

df_backup = df.copy() # saving for later use of original df


In [27]:

# QA/QC Checks

# 1. Check for duplicate variable names
duplicate_columns = df.columns[df.columns.duplicated()].tolist()
if duplicate_columns:
    print(f"Duplicate column names found: {duplicate_columns}")
else:
    print("No duplicate variable names found.")


No duplicate variable names found.


In [28]:

# 2. Check if timestamp columns exist
expected_timestamps = ["TIMESTAMP_START", "TIMESTAMP_END"]
missing_timestamps = [col for col in expected_timestamps if col not in df.columns]
if missing_timestamps:
    print(f"Missing timestamp columns: {missing_timestamps}")
else:
    print("All required timestamp columns are present.")


All required timestamp columns are present.


In [29]:

# 3. Validate timestamp format
for col in expected_timestamps:
    if col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col], format="%Y%m%d%H%M")
            print(f"Timestamps in {col} are in the correct format.")
        except Exception as e:
            print(f"Timestamp format error in {col}: {e}")


Timestamps in TIMESTAMP_START are in the correct format.
Timestamps in TIMESTAMP_END are in the correct format.


In [30]:

# 4. Check for missing values
missing_values = df.isnull().sum()
missing_vars = missing_values[missing_values > 0]
if not missing_vars.empty:
    print(f"NAN values found in columns:\n{missing_vars}")
else:
    print("No NAN values found.")


No NAN values found.


In [31]:

# Convert timestamp columns to datetime
df = df.sort_values(by="TIMESTAMP_START")
df["START"] = pd.to_datetime(df["TIMESTAMP_START"], format="%Y%m%d%H%M")
df["END"] = pd.to_datetime(df["TIMESTAMP_END"], format="%Y%m%d%H%M")

print(f"start time is {df['START'].iloc[0]} and end time is {df['END'].iloc[-1]}")
period =  df['END'].iloc[-1] - df['START'].iloc[0]
print(f"time difference is {period}")
exp_num_of_rows = period/pd.Timedelta(minutes=30) 
num_of_rows = len(df)
if exp_num_of_rows == num_of_rows:
    print("number of rows are correct. No missing timesteps")
else:
    missing_rows = exp_num_of_rows - num_of_rows
    print(f"number of rows are not correct. Missing {int(missing_rows)} timesteps.")



start time is 2024-09-01 00:00:00 and end time is 2024-10-01 00:00:00
time difference is 30 days 00:00:00
number of rows are not correct. Missing 1 timesteps.


In [34]:

# Calculate expected time difference
expected_time_diff = pd.Timedelta(minutes=30)

# 1. Check for time gaps between consecutive rows
df["time_diff"] = df["TIMESTAMP_START"].diff()
gap_rows = df[df["time_diff"] != expected_time_diff]
if not gap_rows.empty:
    print(f"Time gaps detected in data:\n{gap_rows[['TIMESTAMP_START', 'time_diff']]}")


Time gaps detected in data:
         TIMESTAMP_START       time_diff
0    2024-09-01 00:00:00             NaT
1421 2024-09-30 15:00:00 0 days 01:00:00


In [35]:

# 2. Check if (Last TIMESTAMP_END - First TIMESTAMP_START) / Total Rows = 30 min
total_duration = df["TIMESTAMP_END"].iloc[-1] - df["TIMESTAMP_START"].iloc[0]
average_interval = total_duration / (len(df) - 1)  # -1 to get the intervals
if average_interval != expected_time_diff:
    print(f"Average time interval is {average_interval}, which is not 30 minutes.")
else:
    print("Time interval check passed: 30 minutes per row.")

print("Time step checks completed.")



Average time interval is 0 days 00:30:02.503477051, which is not 30 minutes.
Time step checks completed.


In [50]:
# 6. Check if important AmeriFlux variables are present
ameriflux_vars_imp = [
    "FETCH_70", "FETCH_80", "FETCH_90", "FETCH_MAX", "CH4", "CH4_MIXING_RATIO", "CO", "CO2",
    "CO2_MIXING_RATIO", "CO2_SIGMA", "CO2C13", "FC", "FCH4", "FN2O", "FNO", "FNO2", "FO3", "H2O",
    "H2O_MIXING_RATIO", "H2O_SIGMA", "N2O", "N2O_MIXING_RATION", "NO", "NO2", "O3", "SC", "SCH4",
    "SN2O", "SNO", "SNO2", "SO2", "SO3", "FH2O", "G", "H", "LE", "SB", "SG", "SH", "SLE", "PA",
    "RH", "T_SONIC", "T_SONIC_SIGMA", "TA", "VPD", "SW_IN", "SW_OUT", "PA", "TA", "RH", "VPD", "SC", "SH", "SLE"
]


missing_vars = [var for var in ameriflux_vars_imp if var not in df.columns]
if missing_vars:
    print(f"Missing Some AmeriFlux variables, which may be required: {missing_vars}")
else:
    print("All required AmeriFlux variables are present.")


Missing Some AmeriFlux variables, which may be required: ['CH4', 'CH4_MIXING_RATIO', 'CO', 'CO2_SIGMA', 'CO2C13', 'FCH4', 'FN2O', 'FNO', 'FNO2', 'FO3', 'H2O_SIGMA', 'N2O', 'N2O_MIXING_RATION', 'NO', 'NO2', 'O3', 'SC', 'SCH4', 'SN2O', 'SNO', 'SNO2', 'SO2', 'SO3', 'G', 'SB', 'SG', 'SH', 'SLE', 'SW_IN', 'SW_OUT', 'SC', 'SH', 'SLE']


In [54]:

# 6. Check if all required AmeriFlux variables are present
ameriflux_vars = [
    "TIMESTAMP_START", "TIMESTAMP_END", "TIMESTAMP", "COND_WATER", "DO", "PCH4", "PCO2", "PN2O",
    "PPFD_UW_IN", "TW", "DBH", "LEAF_WET", "SAP_DT", "SAP_FLOW", "T_BOLE", "T_CANOPY", "FETCH_70",
    "FETCH_80", "FETCH_90", "FETCH_FILTER", "FETCH_MAX", "CH4", "CH4_MIXING_RATIO", "CO", "CO2",
    "CO2_MIXING_RATIO", "CO2_SIGMA", "CO2C13", "FC", "FCH4", "FN2O", "FNO", "FNO2", "FO3", "H2O",
    "H2O_MIXING_RATIO", "H2O_SIGMA", "N2O", "N2O_MIXING_RATION", "NO", "NO2", "O3", "SC", "SCH4",
    "SN2O", "SNO", "SNO2", "SO2", "SO3", "FH2O", "G", "H", "LE", "SB", "SG", "SH", "SLE", "PA",
    "PBLH", "RH", "T_SONIC", "T_SONIC_SIGMA", "TA", "VPD", "D_SNOW", "P", "P_RAIN", "P_SNOW",
    "RUNOFF", "STEMFLOW", "THROUGHFALL", "ALB", "APAR", "EVI", "FAPAR", "FIPAR", "LW_BC_IN",
    "LW_BC_OUT", "LW_IN", "LW_OUT", "MCRI", "MTCI", "NDVI", "NETRAD", "NIRV", "PPFD_BC_IN",
    "PPFD_BC_OUT", "PPFD_DIF", "PPFD_DIR", "PPFD_IN", "PPFD_OUT", "PRI", "R_UVA", "R_UVB",
    "REDCIRed", "REP", "SPEC_NIR_IN", "SPEC_NIR_OUT", "SPEC_NIR_REFL", "SPEC_PRI_REF_IN",
    "SPEC_PRI_REF_OUT", "SPEC_PRI_REF_REFL", "SPEC_PRI_TGT_IN", "SPEC_PRI_TGT_OUT",
    "SPEC_PRI_TGT_REFL", "SPEC_RED_IN", "SPEC_RED_OUT", "SPEC_RED_REFL", "SR", "SW_BC_IN",
    "SW_BC_OUT", "SW_DIF", "SW_DIR", "SW_IN", "SW_OUT", "TCARI", "SWC", "SWP", "TS", "TNS", "WTD",
    "MO_LENGTH", "TAU", "U_SIGMA", "USTAR", "V_SIGMA", "W_SIGMA", "WD", "WD_SIGMA", "WS", "WS_MAX",
    "ZL", "GPP", "NEE", "RECO", "FC_SSITC_TEST", "FCH4_SSITC_TEST", "FN2O_SSITC_TEST",
    "FNO_SSITC_TEST", "FNO2_SSITC_TEST", "FO3_SSITC_TEST", "H_SSITC_TEST", "LE_SSITC_TEST",
    "TAU_SSITC_TEST"
]


missing_vars = [var for var in ameriflux_vars if var not in df.columns]
if missing_vars:
    print(f"Missing Some AmeriFlux variables, which may not be required: {missing_vars}")
else:
    print("All required AmeriFlux variables are present.")


Missing Some AmeriFlux variables, which may not be required: ['TIMESTAMP', 'COND_WATER', 'DO', 'PCH4', 'PCO2', 'PN2O', 'PPFD_UW_IN', 'TW', 'DBH', 'LEAF_WET', 'SAP_DT', 'SAP_FLOW', 'T_BOLE', 'T_CANOPY', 'FETCH_FILTER', 'CH4', 'CH4_MIXING_RATIO', 'CO', 'CO2_SIGMA', 'CO2C13', 'FCH4', 'FN2O', 'FNO', 'FNO2', 'FO3', 'H2O_SIGMA', 'N2O', 'N2O_MIXING_RATION', 'NO', 'NO2', 'O3', 'SC', 'SCH4', 'SN2O', 'SNO', 'SNO2', 'SO2', 'SO3', 'G', 'SB', 'SG', 'SH', 'SLE', 'PBLH', 'D_SNOW', 'P', 'P_RAIN', 'P_SNOW', 'RUNOFF', 'STEMFLOW', 'THROUGHFALL', 'ALB', 'APAR', 'EVI', 'FAPAR', 'FIPAR', 'LW_BC_IN', 'LW_BC_OUT', 'LW_IN', 'LW_OUT', 'MCRI', 'MTCI', 'NDVI', 'NETRAD', 'NIRV', 'PPFD_BC_IN', 'PPFD_BC_OUT', 'PPFD_DIF', 'PPFD_DIR', 'PPFD_IN', 'PPFD_OUT', 'PRI', 'R_UVA', 'R_UVB', 'REDCIRed', 'REP', 'SPEC_NIR_IN', 'SPEC_NIR_OUT', 'SPEC_NIR_REFL', 'SPEC_PRI_REF_IN', 'SPEC_PRI_REF_OUT', 'SPEC_PRI_REF_REFL', 'SPEC_PRI_TGT_IN', 'SPEC_PRI_TGT_OUT', 'SPEC_PRI_TGT_REFL', 'SPEC_RED_IN', 'SPEC_RED_OUT', 'SPEC_RED_REFL', 'SR',

In [43]:
# 7. Check for duplicate timestamps
duplicate_timestamps = df["TIMESTAMP_START"].duplicated().sum()
if duplicate_timestamps > 0:
    print(f"Duplicate timestamps found: {duplicate_timestamps}")
else: 
    print("No duplicate timestamps found.")

print("QA/QC checks completed.")

No duplicate timestamps found.
QA/QC checks completed.


In [59]:

# Check which AmeriFlux variables are present
present_vars = [var for var in ameriflux_vars if var in df_backup.columns]
num_of_present_vars = len(present_vars)

# Identify non-AmeriFlux variables
non_ameriflux_vars = [var for var in df_backup.columns if var not in ameriflux_vars]
num_of_non_ameriflux_vars = len(non_ameriflux_vars)

if present_vars:
    print(f"{num_of_present_vars} Standard AmeriFlux variables found: {present_vars}")
else:
    print("No standard AmeriFlux variables found.")

if num_of_non_ameriflux_vars == 0:
    print("All variables in the file are AmeriFlux variables.")
else:
    print(f"{num_of_non_ameriflux_vars} variables in the file are not AmeriFlux variables: {non_ameriflux_vars}")



35 Standard AmeriFlux variables found: ['TIMESTAMP_START', 'TIMESTAMP_END', 'FETCH_70', 'FETCH_80', 'FETCH_90', 'FETCH_MAX', 'CO2', 'CO2_MIXING_RATIO', 'FC', 'H2O', 'H2O_MIXING_RATIO', 'FH2O', 'H', 'LE', 'PA', 'RH', 'T_SONIC', 'T_SONIC_SIGMA', 'TA', 'VPD', 'MO_LENGTH', 'TAU', 'U_SIGMA', 'USTAR', 'V_SIGMA', 'W_SIGMA', 'WD', 'WD_SIGMA', 'WS', 'WS_MAX', 'ZL', 'FC_SSITC_TEST', 'H_SSITC_TEST', 'LE_SSITC_TEST', 'TAU_SSITC_TEST']
All variables in the file are AmeriFlux variables.


In [10]:
# Get the filename for validation
file_name = os.path.basename(file_path)
print(f"Filename: {file_name}")
# 1. Verify Filename Format
filename_pattern = r"^US-\w{3}_HH_\d{12}_\d{12}(?:_[\w\d]+)?\.csv$"
if re.match(filename_pattern, file_name):
    print("✅ Filename format is correct.")
else:
    print("❌ ERROR: Filename format is incorrect!")


Filename: US-CU1_HH_202409010000_202410010000.csv
✅ Filename format is correct.


In [4]:

# 2. Check for Quotes in Variable Names
quoted_columns = [col for col in df.columns if col.startswith('"') or col.endswith('"')]
if quoted_columns:
    print(f"❌ ERROR: Quotes found in column names: {quoted_columns}")
else:
    print("✅ No quotes found in variable names.")


✅ No quotes found in variable names.


In [5]:

# 3. Validate Timestamp Headers
expected_timestamps = ["TIMESTAMP_START", "TIMESTAMP_END"]
missing_timestamps = [col for col in expected_timestamps if col not in df.columns]
if missing_timestamps:
    print(f"❌ ERROR: Missing timestamp headers: {missing_timestamps}")
else:
    print("✅ Timestamp headers are present.")


✅ Timestamp headers are present.


'202410010000.csv'

In [17]:

# 4. Check if Filename Time Components Match File Time Period
try:
    start_time_from_filename = pd.to_datetime(file_name.split("_")[2], format="%Y%m%d%H%M")
    end_time_from_filename = pd.to_datetime(file_name.split("_")[3].split(".")[0], format="%Y%m%d%H%M")
    
    df["TIMESTAMP_START"] = pd.to_datetime(df["TIMESTAMP_START"], format="%Y%m%d%H%M")
    df["TIMESTAMP_END"] = pd.to_datetime(df["TIMESTAMP_END"], format="%Y%m%d%H%M")
    

    if df["TIMESTAMP_START"].iloc[0] == start_time_from_filename and df["TIMESTAMP_END"].iloc[-1] == end_time_from_filename:
        print("✅ Filename time components match file time period.")
    else:
        print("❌ ERROR: Filename time components do NOT match file time period!")
        print(f"Expected Start Time: {start_time_from_filename}, Actual Start Time: {df['TIMESTAMP_START'].iloc[0]}")
        print(f"Expected End Time: {end_time_from_filename}, Actual End Time: {df['TIMESTAMP_END'].iloc[-1]}")
except Exception as e:
    print(f"❌ ERROR in matching filename time components: {e}")


✅ Filename time components match file time period.


In [18]:

# 5. Detect Missing or Gap-Filled Data
missing_values = df.isnull().sum()
missing_vars = missing_values[missing_values > 0].index.tolist()
if missing_vars:
    print(f"⚠️ WARNING: Missing values found in columns: {missing_vars}")
else:
    print("✅ No missing values found.")

all_missing_vars = [col for col in df.columns if df[col].isnull().all()]
if all_missing_vars:
    print(f"❌ ERROR: Variables with ALL missing data: {all_missing_vars}")
else:
    print("✅ No variables have all missing data.")

print("QA/QC checks completed.")


✅ No missing values found.
✅ No variables have all missing data.
QA/QC checks completed.


In [6]:
import pandas as pd
import re
import os
import glob
import logging
from datetime import datetime
data_dir = "/Users/bhupendra/projects/crocus/data/flux_data/data/AmeriFlux/"

# Configure logging

def setup_logger():
    file_name = "Ameriflux_filecheck.log"
    """Setup logger for a specific file."""
    log_file = os.path.join(data_dir, file_name)
    logger = logging.getLogger(log_file)
    logger.setLevel(logging.INFO)
    
    # Clear existing handlers
    if logger.hasHandlers():
        logger.handlers.clear()
    
    # Create handlers
    handler = logging.FileHandler(log_file, mode='w')
    handler.setFormatter(logging.Formatter('%(message)s'))
    logger.addHandler(handler)
    
    # Also print logs to console
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(logging.Formatter('%(message)s'))
    logger.addHandler(console_handler)
    
    return logger


def check_filename(file_name, logger):
    """Check if the filename matches the expected AmeriFlux format."""
    logger.info("### Filename Check")
    filename_pattern = r"^US-\w{3}_HH_\d{12}_\d{12}(?:_[\w\d]+)?\.csv$"
    if re.match(filename_pattern, file_name):
        logger.info("✅ Filename format is correct.")
    else:
        logger.error("❌ ERROR: Filename format is incorrect!")
    logger.info("")

def check_variable_headers(df, logger):
    """Check for duplicate and quoted variable names."""
    logger.info("### Variable Header Checks")
    
    # Duplicate column names
    duplicate_columns = df.columns[df.columns.duplicated()].tolist()
    if duplicate_columns:
        logger.warning(f"⚠️ WARNING: Duplicate column names found: {duplicate_columns}")
    else:
        logger.info("✅ No duplicate variable names found.")

    # Check for quotes in variable names
    quoted_columns = [col for col in df.columns if col.startswith('"') or col.endswith('"')]
    if quoted_columns:
        logger.error(f"❌ ERROR: Quotes found in column names: {quoted_columns}")
    else:
        logger.info("✅ No quotes found in variable names.")
    logger.info("")

def check_timestamp_headers(df, logger):
    """Validate timestamp headers and format."""
    logger.info("### Timestamp Header Checks")
    
    expected_timestamps = ["TIMESTAMP_START", "TIMESTAMP_END"]
    missing_timestamps = [col for col in expected_timestamps if col not in df.columns]
    
    if missing_timestamps:
        logger.error(f"❌ ERROR: Missing timestamp headers: {missing_timestamps}")
    else:
        logger.info("✅ All required timestamp columns are present.")

        # Check format
        for col in expected_timestamps:
            try:
                df[col] = pd.to_datetime(df[col], format="%Y%m%d%H%M")
                logger.info(f"✅ Timestamps in {col} are in the correct format.")
            except Exception as e:
                logger.error(f"❌ ERROR: Timestamp format error in {col}: {e}")
    logger.info("")

def check_filename_time_consistency(df, file_name, logger):
    """Check if the filename time components match the file's time period."""
    logger.info("### Filename Time Consistency Check")

    try:
        start_time_from_filename = pd.to_datetime(file_name.split("_")[2], format="%Y%m%d%H%M")
        end_time_from_filename = pd.to_datetime(file_name.split("_")[3].split(".")[0], format="%Y%m%d%H%M")

        actual_start_time = df["TIMESTAMP_START"].iloc[0]
        actual_end_time = df["TIMESTAMP_END"].iloc[-1]

        if actual_start_time == start_time_from_filename and actual_end_time == end_time_from_filename:
            logger.info("✅ Filename time components match file time period.")
        else:
            logger.error("❌ ERROR: Filename time components do NOT match file time period!")
            logger.error(f"Expected Start Time: {start_time_from_filename}, Actual Start Time: {actual_start_time}")
            logger.error(f"Expected End Time: {end_time_from_filename}, Actual End Time: {actual_end_time}")
    except Exception as e:
        logger.error(f"❌ ERROR in matching filename time components: {e}")
    logger.info("")

def check_missing_values(df, logger):
    """Check for missing values in the dataset."""
    logger.info("### Missing Data Check")
    
    missing_values = df.isnull().sum()
    missing_vars = missing_values[missing_values > 0].index.tolist()
    
    if missing_vars:
        logger.warning(f"⚠️ WARNING: Missing values found in columns: {missing_vars}")
    else:
        logger.info("✅ No missing values found.")

    all_missing_vars = [col for col in df.columns if df[col].isnull().all()]
    if all_missing_vars:
        logger.error(f"❌ ERROR: Variables with ALL missing data: {all_missing_vars}")
    else:
        logger.info("✅ No variables have all missing data.")
    logger.info("")

def check_time_gaps(df, logger):
    """Check for time gaps between consecutive rows."""
    logger.info("### Time Gap Check")

    expected_time_diff = pd.Timedelta(minutes=30)
    df["time_diff"] = df["TIMESTAMP_START"].diff()
    gap_rows = df[df["time_diff"] != expected_time_diff]

    if not gap_rows.empty:
        logger.warning(f"⚠️ WARNING: Time gaps detected in data:\n{gap_rows[['TIMESTAMP_START', 'time_diff']]}")
    else:
        logger.info("✅ No time gaps detected.")
    logger.info("")

def check_required_variables(df, logger):
    """Check if all required AmeriFlux variables are present."""
    logger.info("### AmeriFlux Variable Check")

    ameriflux_vars = ["CO2", "H2O", "LE", "H", "TA", "PA", "RH", "VPD", "FC"]
    missing_vars = [var for var in ameriflux_vars if var not in df.columns]

    if missing_vars:
        logger.warning(f"⚠️ WARNING: Missing Some Required AmeriFlux Variables: {missing_vars}")
    else:
        logger.info("✅ All required AmeriFlux variables are present.")
    logger.info("")

def process_file(file_path):
    """Process a single AmeriFlux CSV file and run QA/QC checks."""
    file_name = os.path.basename(file_path)

    logger.info(f"# QA/QC Report for **{file_name}**")
    logger.info(f"_Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}_\n")

    df = pd.read_csv(file_path)

    check_filename(file_name, logger)
    check_variable_headers(df, logger)
    check_timestamp_headers(df, logger)
    check_filename_time_consistency(df, file_name, logger)
    check_missing_values(df, logger)
    check_time_gaps(df, logger)
    check_required_variables(df, logger)

    logger.info("✅ **QA/QC Checks Completed Successfully** ✅\n")
    logger.info("---\n")
def process_directory(directory):
    """Process all AmeriFlux CSV files in a directory."""
    csv_files = glob.glob(os.path.join(directory, "*.csv"))
    
    if not csv_files:
        print("❌ No CSV files found in the directory!")
        return
    
    for file_path in csv_files:
        process_file(file_path)

# Run QA/QC on a directory
logger = setup_logger()
process_directory(data_dir)
logger.handlers.clear()  # Close the logger after processing



# QA/QC Report for **US-CU1_HH_202410010000_202411010000.csv**
_Generated on 2025-02-22 18:11:28_

### Filename Check
✅ Filename format is correct.

### Variable Header Checks
✅ No duplicate variable names found.
✅ No quotes found in variable names.

### Timestamp Header Checks
✅ All required timestamp columns are present.
✅ Timestamps in TIMESTAMP_START are in the correct format.
✅ Timestamps in TIMESTAMP_END are in the correct format.

### Filename Time Consistency Check
✅ Filename time components match file time period.

### Missing Data Check
✅ No missing values found.
✅ No variables have all missing data.

### Time Gap Check
⚠️ WARNING: Time gaps detected in data:
         TIMESTAMP_START       time_diff
0    2024-10-01 00:00:00             NaT
91   2024-10-02 22:00:00 0 days 01:00:00
458  2024-10-10 14:00:00 0 days 01:00:00
1246 2024-10-27 00:30:00 0 days 01:00:00

### AmeriFlux Variable Check
✅ All required AmeriFlux variables are present.

✅ **QA/QC Checks Completed Successfull